In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf

2024-04-16 15:35:49.607638: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-16 15:35:49.708448: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-16 15:35:49.921710: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-16 15:35:53.017755: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Data

In [2]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()

(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist

X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

X_train, X_valid, X_test = X_train/255, X_valid/255, X_test/255

class_names = ["T-shirt/top", "Trouser", "Pullover", 
               "Dress", "Coat", "Sandal", "Shirt",
               "Sneaker", "Bag", "Ankle boot"]

In [3]:
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)

X_train_scaled = (X_train - pixel_means)/pixel_stds
X_valid_scaled = (X_valid - pixel_means)/pixel_stds
X_test_scaled = (X_test - pixel_means)/pixel_stds

# Activation functions

## relu

In [ ]:
tf.random.set_seed(2024)

model = tf.keras.Sequential()

model.add(tf.keras.layers.Flatten(input_shape=[28, 28]))

for layer in range(100):
    model.add(tf.keras.layers.Dense(100, activation="relu",
                                    kernel_initializer = "lecun_normal"))

model.add(tf.keras.layers.Dense(10, activation="softmax"))


In [ ]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
    metrics = ["accuracy"]
)

In [ ]:
history = model.fit(X_train_scaled, y_train, epochs=5,
                    validation_data= (X_valid_scaled, y_valid))

# elu / selu

In [ ]:
tf.random.set_seed(2024)

model = tf.keras.Sequential()

model.add(tf.keras.layers.Flatten(input_shape=[28, 28]))

for layer in range(100):
    model.add(tf.keras.layers.Dense(100, activation="selu", # elu
                                    kernel_initializer = "lecun_normal"))

model.add(tf.keras.layers.Dense(10, activation="softmax"))

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
    metrics = ["accuracy"]
)

history = model.fit(X_train_scaled, y_train, epochs=5,
                    validation_data= (X_valid_scaled, y_valid))

# Batch Normalization

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(2024)

model1 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, activation="relu"),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(200, activation="relu"),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")    
])

model1.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
    metrics = ["accuracy"]
)

history = model1.fit(X_train_scaled, y_train, epochs=5,
                    validation_data= (X_valid_scaled, y_valid))


# Gradient Clipping

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, clipvalue=1.0)

model1.compile(
    loss="sparse_categorical_crossentropy",
    optimizer= optimizer,
    metrics = ["accuracy"]
)

history = model1.fit(X_train_scaled, y_train, epochs=5,
                    validation_data= (X_valid_scaled, y_valid))
